In [14]:
import zipfile
import glob
import os

ZIP_DIR = "地形"   # zipが入っているフォルダ
OUT_DIR = "地形-解凍"   # 解凍先フォルダ

os.makedirs(OUT_DIR, exist_ok=True)

zip_files = glob.glob(os.path.join(ZIP_DIR, "*.zip"))
print("見つかったZIPファイル数:", len(zip_files))

for z in zip_files:
    print("解凍中:", z)
    with zipfile.ZipFile(z, 'r') as zip_ref:
        # zipの中身を OUT_DIR 配下に展開
        zip_ref.extractall(OUT_DIR)

print("✅ 解凍完了しました！")


見つかったZIPファイル数: 174
解凍中: 地形\SDDSWS3036.zip
解凍中: 地形\SDDSWS3622.zip
解凍中: 地形\SDDSWS3623.zip
解凍中: 地形\SDDSWS3624.zip
解凍中: 地形\SDDSWS3631.zip
解凍中: 地形\SDDSWS3641.zip
解凍中: 地形\SDDSWS3653.zip
解凍中: 地形\SDDSWS3724.zip
解凍中: 地形\SDDSWS3725.zip
解凍中: 地形\SDDSWS3741.zip
解凍中: 地形\SDDSWS3823.zip
解凍中: 地形\SDDSWS3824.zip
解凍中: 地形\SDDSWS3831.zip
解凍中: 地形\SDDSWS3841.zip
解凍中: 地形\SDDSWS3926.zip
解凍中: 地形\SDDSWS3927.zip
解凍中: 地形\SDDSWS3928.zip
解凍中: 地形\SDDSWS3942.zip
解凍中: 地形\SDDSWS4027.zip
解凍中: 地形\SDDSWS4028.zip
解凍中: 地形\SDDSWS4040.zip
解凍中: 地形\SDDSWS4042.zip
解凍中: 地形\SDDSWS4128.zip
解凍中: 地形\SDDSWS4129.zip
解凍中: 地形\SDDSWS4142.zip
解凍中: 地形\SDDSWS4229.zip
解凍中: 地形\SDDSWS4230.zip
解凍中: 地形\SDDSWS4328.zip
解凍中: 地形\SDDSWS4329.zip
解凍中: 地形\SDDSWS4429.zip
解凍中: 地形\SDDSWS4440.zip
解凍中: 地形\SDDSWS4530.zip
解凍中: 地形\SDDSWS4531.zip
解凍中: 地形\SDDSWS4540.zip
解凍中: 地形\SDDSWS4629.zip
解凍中: 地形\SDDSWS4630.zip
解凍中: 地形\SDDSWS4631.zip
解凍中: 地形\SDDSWS4728.zip
解凍中: 地形\SDDSWS4729.zip
解凍中: 地形\SDDSWS4730.zip
解凍中: 地形\SDDSWS4731.zip
解凍中: 地形\SDDSWS4739.zip
解凍中: 地形\SDDSWS4

In [11]:
import geopandas as gpd
import pandas as pd
import glob

# 1. mesh_shp 以下の .shp を全部探す（再帰）
shp_files = glob.glob("地形-解凍/**/*.shp", recursive=True)
print("見つかった Shapefile の数:", len(shp_files))
print(shp_files[:5])  # 最初の5件だけ表示

if not shp_files:
    raise FileNotFoundError("Shapefile が見つかりません。解凍フォルダやパスを確認してください。")

gdf_list = []
for f in shp_files:
    print("読み込み中:", f)
    gdf = gpd.read_file(f)
    gdf_list.append(gdf)

# 2. 結合
gdf_all = pd.concat(gdf_list, ignore_index=True)
print("結合後のレコード数:", len(gdf_all))

# 3. 列名を確認して、人口列を特定する
print("列名一覧:")
print(gdf_all.columns)

# ↓ ここを実際の人口の列名に変える（例："人口総数", "総人口", "P_NUM" など）
POP_COL = "人口総数"

# 4. 座標系をWGS84にそろえる
if gdf_all.crs is None:
    gdf_all = gdf_all.set_crs(epsg=4326)
else:
    gdf_all = gdf_all.to_crs(epsg=4326)

# 5. セントロイドを計算（メッシュの中心）
centroids = gdf_all.geometry.centroid
lon = centroids.x
lat = centroids.y

df_out = pd.DataFrame({
    "lon": lon,
    "lat": lat,
    "population": gdf_all[POP_COL].astype(float)
})

df_out.to_csv("japan_population_1km.csv", index=False, encoding="utf-8")
print("✅ japan_population_1km.csv を出力しました")


見つかった Shapefile の数: 170
['地形-解凍\\MESH03036.shp', '地形-解凍\\MESH03622.shp', '地形-解凍\\MESH03623.shp', '地形-解凍\\MESH03624.shp', '地形-解凍\\MESH03631.shp']
読み込み中: 地形-解凍\MESH03036.shp
読み込み中: 地形-解凍\MESH03622.shp
読み込み中: 地形-解凍\MESH03623.shp
読み込み中: 地形-解凍\MESH03624.shp
読み込み中: 地形-解凍\MESH03631.shp
読み込み中: 地形-解凍\MESH03641.shp
読み込み中: 地形-解凍\MESH03653.shp
読み込み中: 地形-解凍\MESH03724.shp
読み込み中: 地形-解凍\MESH03725.shp
読み込み中: 地形-解凍\MESH03741.shp
読み込み中: 地形-解凍\MESH03823.shp
読み込み中: 地形-解凍\MESH03824.shp
読み込み中: 地形-解凍\MESH03831.shp
読み込み中: 地形-解凍\MESH03841.shp
読み込み中: 地形-解凍\MESH03926.shp
読み込み中: 地形-解凍\MESH03927.shp
読み込み中: 地形-解凍\MESH03928.shp
読み込み中: 地形-解凍\MESH03942.shp
読み込み中: 地形-解凍\MESH04027.shp
読み込み中: 地形-解凍\MESH04028.shp
読み込み中: 地形-解凍\MESH04040.shp
読み込み中: 地形-解凍\MESH04042.shp
読み込み中: 地形-解凍\MESH04128.shp
読み込み中: 地形-解凍\MESH04129.shp
読み込み中: 地形-解凍\MESH04142.shp
読み込み中: 地形-解凍\MESH04229.shp
読み込み中: 地形-解凍\MESH04230.shp
読み込み中: 地形-解凍\MESH04328.shp
読み込み中: 地形-解凍\MESH04329.shp
読み込み中: 地形-解凍\MESH04429.shp
読み込み中: 地形-解凍\MESH04440.shp
読み込み中: 地形-解凍\MESH0

KeyboardInterrupt: 

In [2]:
import zipfile
import glob
import os

ZIP_DIR = "population"   # ZIPファイルの置き場所
OUT_DIR = "population解凍"   # 解凍先のフォルダ

# 出力フォルダが無ければ作成
os.makedirs(OUT_DIR, exist_ok=True)

# フォルダ内の .zip を全部取得
zip_files = glob.glob(os.path.join(ZIP_DIR, "*.zip"))
print("見つかったZIPファイル数:", len(zip_files))

for z in zip_files:
    print("解凍中:", z)
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall(OUT_DIR)

print("🎉 全ZIPの解凍が完了しました！")


見つかったZIPファイル数: 47
解凍中: population\tblT001100S01.zip
解凍中: population\tblT001100S02.zip
解凍中: population\tblT001100S03.zip
解凍中: population\tblT001100S04.zip
解凍中: population\tblT001100S05.zip
解凍中: population\tblT001100S06.zip
解凍中: population\tblT001100S07.zip
解凍中: population\tblT001100S08.zip
解凍中: population\tblT001100S09.zip
解凍中: population\tblT001100S10.zip
解凍中: population\tblT001100S11.zip
解凍中: population\tblT001100S12.zip
解凍中: population\tblT001100S13.zip
解凍中: population\tblT001100S14.zip
解凍中: population\tblT001100S15.zip
解凍中: population\tblT001100S16.zip
解凍中: population\tblT001100S17.zip
解凍中: population\tblT001100S18.zip
解凍中: population\tblT001100S19.zip
解凍中: population\tblT001100S20.zip
解凍中: population\tblT001100S21.zip
解凍中: population\tblT001100S22.zip
解凍中: population\tblT001100S23.zip
解凍中: population\tblT001100S24.zip
解凍中: population\tblT001100S25.zip
解凍中: population\tblT001100S26.zip
解凍中: population\tblT001100S27.zip
解凍中: population\tblT001100S28.zip
解凍中: population\tblT001100S29.

In [6]:
import pandas as pd

df = pd.read_csv("population解凍/tblT001100S01.txt",encoding='shift_jis')  # ファイル名に変える
print(df.head())
print(df.columns)


     KEY_CODE  HTKSYORI     HTKSAKI    GASSAN T001100001 T001100002  \
0         NaN       NaN         NaN       NaN    　人口（総数）  　人口（総数）　男   
1  62392749.0       2.0  62392759.0       NaN          7          4   
2  62392759.0       1.0         NaN  62392749        631        280   
3  62392769.0       0.0         NaN       NaN        165         73   
4  62393718.0       0.0         NaN       NaN         45         16   

  T001100003   T001100004  T001100005  T001100006  ...    T001100041  \
0  　人口（総数）　女  　０〜１４歳人口　総数  　０〜１４歳人口　男  　０〜１４歳人口　女  ...  　６人世帯数　一般世帯数   
1          3            *           *           *  ...             *   
2        351           14           8           6  ...             1   
3         92           14           5           9  ...             0   
4         29            1           0           1  ...             0   

       T001100042       T001100043     T001100044        T001100045  \
0  　７人以上世帯数　一般世帯数  　親族のみの世帯数　一般世帯数  　核家族世帯数　一般世帯数  　核家族以外の世帯数　一般世帯数  

C:\Users\moko0\AppData\Local\Temp\ipykernel_45604\3378006275.py:3: DtypeWarning: Columns (4,5,6,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("population解凍/tblT001100S01.txt",encoding='shift_jis')  # ファイル名に変える


In [8]:
import pandas as pd
import glob

# 1. TXTファイルを全部読む
txt_files = glob.glob("population解凍/*.txt")
print("TXTファイル数:", len(txt_files))

df_list = []
for f in txt_files:
    print("読み込み:", f)
    # 文字コードはだいたい shift_jis
    temp = pd.read_csv(f, encoding="shift_jis")
    
    # 行0 は「人口（総数）」などの日本語ラベルなので捨てる
    temp = temp.iloc[1:].copy()
    
    # 総人口列 T001100001 を数値に変換（* は NaN → 0 にしておく）
    temp["T001100001"] = (
        pd.to_numeric(temp["T001100001"], errors="coerce")
          .fillna(0)
          .astype(int)
    )
    
    df_list.append(temp[["KEY_CODE", "T001100001"]])

# 2. 結合
df_pop = pd.concat(df_list, ignore_index=True)
print("結合後レコード数:", len(df_pop))

# 3. 列名を分かりやすく
df_pop = df_pop.rename(columns={
    "KEY_CODE": "meshcode",
    "T001100001": "population"
})

# 4. 保存
df_pop.to_csv("population_all.csv", index=False, encoding="utf-8")
print("✅ population_all.csv を出力しました（meshcode, population）")


TXTファイル数: 47
読み込み: population解凍\tblT001100S01.txt


C:\Users\moko0\AppData\Local\Temp\ipykernel_45604\115543368.py:12: DtypeWarning: Columns (4,5,6,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(f, encoding="shift_jis")


読み込み: population解凍\tblT001100S02.txt
読み込み: population解凍\tblT001100S03.txt
読み込み: population解凍\tblT001100S04.txt
読み込み: population解凍\tblT001100S05.txt
読み込み: population解凍\tblT001100S06.txt
読み込み: population解凍\tblT001100S07.txt
読み込み: population解凍\tblT001100S08.txt
読み込み: population解凍\tblT001100S09.txt
読み込み: population解凍\tblT001100S10.txt
読み込み: population解凍\tblT001100S11.txt
読み込み: population解凍\tblT001100S12.txt
読み込み: population解凍\tblT001100S13.txt
読み込み: population解凍\tblT001100S14.txt
読み込み: population解凍\tblT001100S15.txt
読み込み: population解凍\tblT001100S16.txt
読み込み: population解凍\tblT001100S17.txt
読み込み: population解凍\tblT001100S18.txt
読み込み: population解凍\tblT001100S19.txt
読み込み: population解凍\tblT001100S20.txt
読み込み: population解凍\tblT001100S21.txt
読み込み: population解凍\tblT001100S22.txt
読み込み: population解凍\tblT001100S23.txt
読み込み: population解凍\tblT001100S24.txt
読み込み: population解凍\tblT001100S25.txt
読み込み: population解凍\tblT001100S26.txt
読み込み: population解凍\tblT001100S27.txt
読み込み: population解凍\tblT001100S28.txt
読

In [2]:
import os
import glob
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# ==============================
# 設定
# ==============================
MESH_TXT_DIR = "population解凍"   # 人口TXTが入っているフォルダ
MESH_SHP_DIR = "地形-解凍"       # Shapefileが入っているフォルダ
OUTPUT_DIR   = "output"          # 出力フォルダ

os.makedirs(OUTPUT_DIR, exist_ok=True)


# ==============================
# 1. TXT（人口） → population_all.csv
# ==============================
def build_population_table():
    txt_files = glob.glob(os.path.join(MESH_TXT_DIR, "*.txt"))
    print("TXTファイル数:", len(txt_files))
    if not txt_files:
        raise FileNotFoundError("TXTファイルが見つかりませんでした。MESH_TXT_DIR を確認してください。")

    df_list = []

    for f in txt_files:
        print("読み込み:", f)
        # KEY_CODE を最初から文字列にしておく
        temp = pd.read_csv(
            f,
            encoding="shift_jis",
            dtype={"KEY_CODE": str}
        )

        # 行0 は「人口（総数）」などの日本語ラベルなので捨てる
        temp = temp.iloc[1:].copy()

        # T001100001 = 総人口（* は NaN → 0 にする）
        temp["T001100001"] = (
            pd.to_numeric(temp["T001100001"], errors="coerce")
              .fillna(0)
              .astype(int)
        )

        # 必要な列だけ
        temp = temp[["KEY_CODE", "T001100001"]]
        df_list.append(temp)

    df_pop = pd.concat(df_list, ignore_index=True)
    print("人口テーブル 結合後レコード数:", len(df_pop))

    df_pop = df_pop.rename(columns={
        "KEY_CODE": "meshcode",
        "T001100001": "population"
    })

    out_path = os.path.join(OUTPUT_DIR, "population_all.csv")
    df_pop.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ population_all.csv を出力しました → {out_path}")

    return df_pop


# ==============================
# 2. Shapefile → メッシュ中心点（lon, lat）
# ==============================
def build_mesh_points():
    # mesh_shp 以下の .shp を再帰的に取得
    shp_files = glob.glob(os.path.join(MESH_SHP_DIR, "**", "*.shp"), recursive=True)
    print("Shapefile数:", len(shp_files))
    if not shp_files:
        raise FileNotFoundError("Shapefile が見つかりませんでした。MESH_SHP_DIR を確認してください。")

    gdf_list = []
    for f in shp_files:
        print("Shapefile 読み込み:", f)
        gdf = gpd.read_file(f)
        gdf_list.append(gdf)

    gdf_all = pd.concat(gdf_list, ignore_index=True)
    print("Shapefile 結合後レコード数:", len(gdf_all))
    print("列名一覧:", gdf_all.columns.tolist())

    # メッシュコード列名を KEY_CODE と仮定（違う場合はここを変える）
    if "KEY_CODE" in gdf_all.columns:
        gdf_all = gdf_all.rename(columns={"KEY_CODE": "meshcode"})
    else:
        raise KeyError("Shapefile に KEY_CODE 列が見つかりません。メッシュコード列名を確認してください。")

    # meshcode を文字列に統一
    gdf_all["meshcode"] = gdf_all["meshcode"].astype(str)

    # 座標系を WGS84 に揃える
    if gdf_all.crs is None:
        gdf_all = gdf_all.set_crs("EPSG:4326")
    else:
        gdf_all = gdf_all.to_crs("EPSG:4326")

    # メッシュポリゴンの中心点を計算
    centroids = gdf_all.geometry.centroid
    gdf_all["lon"] = centroids.x
    gdf_all["lat"] = centroids.y

    return gdf_all[["meshcode", "lon", "lat"]]


# ==============================
# 3. 位置＋人口を結合 → japan_population_1km.csv
# ==============================
def build_population_points(gdf_points: pd.DataFrame, df_pop: pd.DataFrame):
    # population側の meshcode も文字列に
    df_pop["meshcode"] = df_pop["meshcode"].astype(str)

    # 結合
    merged = gdf_points.merge(df_pop, on="meshcode", how="inner")
    print("位置＋人口 結合後レコード数:", len(merged))

    df_out = merged[["lon", "lat", "population"]]

    out_path = os.path.join(OUTPUT_DIR, "japan_population_1km.csv")
    df_out.to_csv(out_path, index=False, encoding="utf-8")
    print(f"🎉 japan_population_1km.csv を出力しました → {out_path}")

    return df_out


# ==============================
# 4. pydeck で 3D人口密度マップを作成
# ==============================
def build_3d_map(df_points: pd.DataFrame):
    import pydeck as pdk
    pdk.settings.enable_screenshot = True  # ←画像出力のためON

    df = df_points.copy()

    max_pop = df["population"].max()
    df["pop_norm"] = df["population"] / max_pop
    df["height"] = (df["pop_norm"] ** 0.4) * 30000

    intensity = df["pop_norm"] ** 0.5
    df["r"] = (230 + 25 * intensity).clip(0, 255).astype(int)
    df["g"] = (190 - 140 * intensity).clip(40, 255).astype(int)
    df["b"] = (230 + 10 * intensity).clip(0, 255).astype(int)

    # --- ラベル ---
    label_height = 35000
    label_data = pd.DataFrame([
        {"name": "Tokyo",  "lon": 139.6917, "lat": 35.6895},
        {"name": "Osaka",  "lon": 135.5022, "lat": 34.6937},
        {"name": "Aichi",  "lon": 136.9066, "lat": 35.1815},
    ])
    label_data["ground_pos"] = label_data.apply(lambda r: [r["lon"], r["lat"], 0], axis=1)
    label_data["label_pos"]  = label_data.apply(lambda r: [r["lon"], r["lat"], label_height], axis=1)

    # --- レイヤ ---
    column_layer = pdk.Layer(
        "ColumnLayer",
        data=df,
        get_position=["lon", "lat"],
        get_elevation="height",
        radius=750,
        get_fill_color="[r, g, b, 220]",
        pickable=True
    )
    line_layer = pdk.Layer(
        "LineLayer",
        data=label_data,
        get_source_position="ground_pos",
        get_target_position="label_pos",
        get_width=3,
        get_color=[80, 80, 80, 230]
    )
    text_layer = pdk.Layer(
        "TextLayer",
        data=label_data,
        get_position="label_pos",
        get_text="name",
        get_size=28,
        billboard=True,
        get_color=[20, 20, 20, 255],
        size_units="meters"
    )

    view_state = pdk.ViewState(
        longitude=137.0,
        latitude=36.0,
        zoom=4,
        pitch=60
    )

    deck = pdk.Deck(
        layers=[column_layer, line_layer, text_layer],
        initial_view_state=view_state,
        map_style="light"
    )

    # --- PNGとして書き出す ---
    img_path = os.path.join(OUTPUT_DIR, "japan_3d_population_labels.png")
    deck.to_image(img_path)
    print(f"📸 PNG画像を出力しました → {img_path}")


# ==============================
# メイン処理
# ==============================
if __name__ == "__main__":
    print("=== 1. population_all.csv を作成 ===")
    df_pop = build_population_table()

    print("\n=== 2. メッシュ中心点を作成 ===")
    gdf_points = build_mesh_points()

    print("\n=== 3. 位置＋人口 を結合 ===")
    df_points = build_population_points(gdf_points, df_pop)

    print("\n=== 4. 3D人口密度マップを生成 ===")
    

    print("\n🎊 全処理完了！  output フォルダ内の HTML をブラウザで開いて確認してください。")


=== 1. population_all.csv を作成 ===
TXTファイル数: 47
読み込み: population解凍\tblT001100S01.txt
読み込み: population解凍\tblT001100S02.txt


C:\Users\moko0\AppData\Local\Temp\ipykernel_26864\561679322.py:31: DtypeWarning: Columns (4,5,6,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(


読み込み: population解凍\tblT001100S03.txt
読み込み: population解凍\tblT001100S04.txt
読み込み: population解凍\tblT001100S05.txt
読み込み: population解凍\tblT001100S06.txt
読み込み: population解凍\tblT001100S07.txt
読み込み: population解凍\tblT001100S08.txt
読み込み: population解凍\tblT001100S09.txt
読み込み: population解凍\tblT001100S10.txt
読み込み: population解凍\tblT001100S11.txt
読み込み: population解凍\tblT001100S12.txt
読み込み: population解凍\tblT001100S13.txt
読み込み: population解凍\tblT001100S14.txt
読み込み: population解凍\tblT001100S15.txt
読み込み: population解凍\tblT001100S16.txt
読み込み: population解凍\tblT001100S17.txt
読み込み: population解凍\tblT001100S18.txt
読み込み: population解凍\tblT001100S19.txt
読み込み: population解凍\tblT001100S20.txt
読み込み: population解凍\tblT001100S21.txt
読み込み: population解凍\tblT001100S22.txt
読み込み: population解凍\tblT001100S23.txt
読み込み: population解凍\tblT001100S24.txt
読み込み: population解凍\tblT001100S25.txt
読み込み: population解凍\tblT001100S26.txt
読み込み: population解凍\tblT001100S27.txt
読み込み: population解凍\tblT001100S28.txt
読み込み: population解凍\tblT001100S29.txt
読

C:\Users\moko0\AppData\Local\Temp\ipykernel_26864\561679322.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gdf_all.geometry.centroid



=== 3. 位置＋人口 を結合 ===
位置＋人口 結合後レコード数: 177859
🎉 japan_population_1km.csv を出力しました → output\japan_population_1km.csv

=== 4. 3D人口密度マップを生成 ===

🎊 全処理完了！  output フォルダ内の HTML をブラウザで開いて確認してください。


In [5]:
import os
import numpy as np
import pandas as pd
import pydeck as pdk

OUTPUT_DIR = "output"
csv_path = os.path.join(OUTPUT_DIR, "japan_population_1km.csv")

df = pd.read_csv(csv_path)

# ==============================
# 1. 高さを「バカ高く」＆尖らせる
# ==============================
max_pop = df["population"].max()
df["pop_norm"] = df["population"] / max_pop  # 0〜1

# 低いところをつぶして、都市だけドーンと伸ばす
# 0.25〜0.4くらいの指数で調整。小さいほど尖る
df["height"] = (df["pop_norm"] ** 0.28) * 60000  # 倍率 60000 はかなり高め

# ==============================
# 2. ネオン水色グラデーションの色をつける
# ==============================
# intensity を 0〜1 で作成（少し強調）
intensity = df["pop_norm"] ** 0.35

# 暗いところ = 暗い青 / 明るいところ = ほぼ白っぽいシアン
df["r"] = (40  + 180 * intensity).clip(0, 255).astype(int)   # 40〜220
df["g"] = (160 +  90 * intensity).clip(0, 255).astype(int)   # 160〜250
df["b"] = (200 +  55 * intensity).clip(0, 255).astype(int)   # 200〜255

# ==============================
# 3. ビュー設定（斜め上から、山を強調）
# ==============================
view_state = pdk.ViewState(
    longitude=137.0,
    latitude=36.0,
    zoom=4.2,    # もう少し寄せたかったら 4.5 とか
    pitch=70,    # 立ち上がり強め
    bearing=25,  # 少し回転させて奥行きを出す
)

# ==============================
# 4. ColumnLayer（細く高いネオンピラー）
# ==============================
column_layer = pdk.Layer(
    "ColumnLayer",
    data=df,
    get_position=["lon", "lat"],
    get_elevation="height",
    elevation_scale=1.0,
    radius=220,                         # 細めにしてトゲ感UP
    get_fill_color="[r, g, b, 230]",    # ネオン水色＋半透明
    pickable=False,
    auto_highlight=False,
)

# ==============================
# 5. Deck 作成（背景を真っ黒に）
# ==============================
deck = pdk.Deck(
    layers=[column_layer],
    initial_view_state=view_state,            # ベースマップなし
    map_style="mapbox://styles/mapbox/dark-v10", # 真っ黒背景
)

html_path = os.path.join(OUTPUT_DIR, "japan_3d_neon_population.html")
deck.to_html(html_path, notebook_display=False)
print(f"✅ {html_path} を出力しました")


✅ output\japan_3d_neon_population.html を出力しました


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.io as pio

# === 1. データ読み込み ===
CSV_PATH = "SSDSE-B-2025.csv"
TARGET_YEAR = 2020

df = pd.read_csv(CSV_PATH)
df_year = df[df["年度"] == TARGET_YEAR].copy()

# === 2. 支出カテゴリ一覧 ===
spend_cols = [
    "食料費（二人以上の世帯）",
    "住居費（二人以上の世帯）",
    "光熱・水道費（二人以上の世帯）",
    "家具・家事用品費（二人以上の世帯）",
    "被服及び履物費（二人以上の世帯）",
    "保健医療費（二人以上の世帯）",
    "交通・通信費（二人以上の世帯）",
    "教育費（二人以上の世帯）",
    "教養娯楽費（二人以上の世帯）",
    "その他の消費支出（二人以上の世帯）"
]

# === 3. K-Means のための標準化 ===
X = df_year[spend_cols].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 4. K-means（3クラスタ） ===
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X_scaled)

df_year["支出クラス"] = labels

# === 5. クラス名の手動付与（より意味のある名前に） ===
# クラスタ中心（平均）から特徴を見て自動判定
centers = kmeans.cluster_centers_
centers_inv = scaler.inverse_transform(centers)
centers_df = pd.DataFrame(centers_inv, columns=spend_cols)

# 各クラスタの特徴で名前付け
rename_map = {}
for idx, row in centers_df.iterrows():
    if row["住居費（二人以上の世帯）"] == max(row["住居費（二人以上の世帯）"] for _, row in centers_df.iterrows()):
        rename_map[idx] = "都市型（家賃中心）"
    elif row["交通・通信費（二人以上の世帯）"] == max(row["交通・通信費（二人以上の世帯）"] for _, row in centers_df.iterrows()):
        rename_map[idx] = "地方中核（車社会）"
    else:
        rename_map[idx] = "低密度（低コスト地域）"

df_year["支出クラス名"] = df_year["支出クラス"].map(rename_map)

# === 6. クラス別平均値 ===
class_mean = df_year.groupby("支出クラス名")[spend_cols].mean().reset_index()

# === 7. 棒グラフ（カテゴリ × クラス） ===
df_long = class_mean.melt(id_vars="支出クラス名", var_name="カテゴリ", value_name="支出額")

fig = px.bar(
    df_long,
    x="カテゴリ",
    y="支出額",
    color="支出クラス名",
    barmode="group",
    title="2020年 支出構造の3クラス（都市型・車社会型・低密度型）",
    labels={
        "支出額": "月あたり支出額（円）",
        "カテゴリ": "支出カテゴリ",
        "支出クラス名": "生活コストモデル"
    }
)

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=120),
    xaxis_tickangle=45
)

fig.show()




c:\Users\moko0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.io as pio

CSV_PATH = "SSDSE-B-2025.csv"
TARGET_YEAR = 2020

df = pd.read_csv(CSV_PATH)
df_year = df[df["年度"] == TARGET_YEAR].copy()

# 支出カテゴリ
spend_cols = [
    "食料費（二人以上の世帯）",
    "住居費（二人以上の世帯）",
    "光熱・水道費（二人以上の世帯）",
    "家具・家事用品費（二人以上の世帯）",
    "被服及び履物費（二人以上の世帯）",
    "保健医療費（二人以上の世帯）",
    "交通・通信費（二人以上の世帯）",
    "教育費（二人以上の世帯）",
    "教養娯楽費（二人以上の世帯）",
    "その他の消費支出（二人以上の世帯）"
]

# 合計支出
df_year["総支出"] = df_year[spend_cols].sum(axis=1)

# 重要：比率特徴量を追加（これで地方中核を明確にする）
df_year["交通比率"] = df_year["交通・通信費（二人以上の世帯）"] / df_year["総支出"]
df_year["住居比率"] = df_year["住居費（二人以上の世帯）"] / df_year["総支出"]
df_year["教育比率"] = df_year["教育費（二人以上の世帯）"] / df_year["総支出"]

# K-means 用データ
X = df_year[spend_cols + ["交通比率", "住居比率", "教育比率"]]

# 標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-means（3クラス）
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X_scaled)

df_year["クラスタ"] = labels

# クラスタ中心の元スケール値
centers = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=X.columns)

# クラス名を自動で付ける
class_names = {}
for idx, row in centers_df.iterrows():
    if row["住居費（二人以上の世帯）"] == centers_df["住居費（二人以上の世帯）"].max():
        class_names[idx] = "都市型（家賃中心）"
    elif row["交通比率"] == centers_df["交通比率"].max():
        class_names[idx] = "地方中核（車社会）"
    else:
        class_names[idx] = "低密度（低コスト）"

df_year["生活コストモデル"] = df_year["クラスタ"].map(class_names)

# クラスごとの平均
plot_df = df_year.groupby("生活コストモデル")[spend_cols].mean().reset_index()

# ロング化
df_long = plot_df.melt(id_vars="生活コストモデル", var_name="カテゴリ", value_name="支出額")

fig = px.bar(
    df_long,
    x="カテゴリ",
    y="支出額",
    color="生活コストモデル",
    barmode="group",
    title="2020年 支出構造の3クラス（都市型・車社会型・低密度型）",
    labels={"支出額": "月あたり支出額（円）", "カテゴリ": "支出カテゴリ"}
)

fig.update_layout(xaxis_tickangle=40)
fig.show()




c:\Users\moko0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

